# Imports

In [1]:
import numpy as np

In [2]:
import os.path as osp
import numpy as np
from tqdm import tqdm
from utils import pickle_save, pickle_load
from utils.data.delf import datum_io
from copy import deepcopy

In [3]:
import sacred
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from numpy import linalg as LA
from utils import pickle_load, pickle_save
#from utils.revisited import compute_metrics
from utils.data.delf import datum_io

In [4]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [5]:
feature_name = 'r50_gldv1'
set_name = 'test'
gnd_name = 'gnd_roxford5k.pkl'

In [6]:
dataset_name = 'oxford5k'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [7]:
use_aqe = False
aqe_params = {'k': 2, 'alpha': 0.3}

save_nn_inds = True

In [22]:
with open(osp.join(data_dir, 'test_query.txt')) as fid:
    query_lines   = fid.read().splitlines()

In [23]:
with open(osp.join(data_dir, set_name+'_gallery.txt')) as fid:
    gallery_lines = fid.read().splitlines()

In [24]:
query_feats = []
for i in tqdm(range(len(query_lines))):
    name = osp.splitext(osp.basename(query_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
    query_feats.append(datum_io.ReadFromFile(path))

100%|██████████| 70/70 [00:00<00:00, 522.72it/s]


In [25]:
query_feats = np.stack(query_feats, axis=0)
query_feats = query_feats / LA.norm(query_feats, axis=-1)[:, None]

In [26]:
index_feats = []
for i in tqdm(range(len(gallery_lines))):
    name = osp.splitext(osp.basename(gallery_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
    index_feats.append(datum_io.ReadFromFile(path))

100%|██████████| 4993/4993 [00:09<00:00, 551.20it/s]


In [27]:
index_feats = np.stack(index_feats, axis=0)
index_feats = index_feats / LA.norm(index_feats, axis=-1)[:, None]

sims = np.matmul(query_feats, index_feats.T)

In [28]:
index_feats.shape, query_feats.shape, sims.shape

((4993, 2048), (70, 2048), (70, 4993))

In [29]:
if use_aqe:
    alpha = aqe_params['alpha']
    nn_inds = np.argsort(-sims, -1)
    query_aug = deepcopy(query_feats)
    for i in range(len(query_feats)):
        new_q = [query_feats[i]]
        for j in range(aqe_params['k']):
            nn_id = nn_inds[i, j]
            weight = sims[i, nn_id] ** aqe_params['alpha']
            new_q.append(weight * index_feats[nn_id])
        new_q = np.stack(new_q, 0)
        new_q = np.mean(new_q, axis=0)
        query_aug[i] = new_q/LA.norm(new_q, axis=-1)
    sims = np.matmul(query_aug, index_feats.T)


In [30]:
nn_inds = np.argsort(-sims, -1)
nn_dists = deepcopy(sims)
for i in range(query_feats.shape[0]):
    for j in range(index_feats.shape[0]):
        nn_dists[i, j] = sims[i, nn_inds[i, j]]

In [31]:
nn_inds.shape, nn_dists.shape, sims.shape

((70, 4993), (70, 4993), (70, 4993))

In [32]:
query_feats.shape[0], index_feats.shape[0], index_feats.shape

(70, 4993, (4993, 2048))

In [35]:
if save_nn_inds:
    output_path = osp.join(data_dir, 'nn_inds_%s.pkl' % feature_name)
    print(output_path)

/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/oxford5k/nn_inds_r50_gldv1.pkl


In [36]:
def compute_ap(ranks, nres):
    """
    Computes average precision for given ranked indexes.
    
    Arguments
    ---------
    ranks : zerro-based ranks of positive images
    nres  : number of positive images
    
    Returns
    -------
    ap    : average precision
    """

    # number of images ranked by the system
    nimgranks = len(ranks)

    # accumulate trapezoids in PR-plot
    ap = 0

    recall_step = 1. / nres

    for j in np.arange(nimgranks):
        rank = ranks[j]

        if rank == 0:
            precision_0 = 1.
        else:
            precision_0 = float(j) / rank

        precision_1 = float(j + 1) / (rank + 1)

        ap += (precision_0 + precision_1) * recall_step / 2.

    return ap

In [54]:
def compute_map(ranks, gnd, kappas=[]):
    """
    Computes the mAP for a given set of returned results.

         Usage: 
           map = compute_map (ranks, gnd) 
                 computes mean average precsion (map) only
        
           map, aps, pr, prs = compute_map (ranks, gnd, kappas) 
                 computes mean average precision (map), average precision (aps) for each query
                 computes mean precision at kappas (pr), precision at kappas (prs) for each query
        
         Notes:
         1) ranks starts from 0, ranks.shape = db_size X #queries
         2) The junk results (e.g., the query itself) should be declared in the gnd stuct array
         3) If there are no positive images for some query, that query is excluded from the evaluation
    """

    map = 0.
    nq = len(gnd) # number of queries
    aps = np.zeros(nq)
    pr = np.zeros(len(kappas))
    prs = np.zeros((nq, len(kappas)))
    nempty = 0

    for i in np.arange(nq):
        qgnd = np.array(gnd[i]['ok'])

        # no positive images, skip from the average
        if qgnd.shape[0] == 0:
            aps[i] = float('nan')
            prs[i, :] = float('nan')
            nempty += 1
            continue

        try:
            qgndj = np.array(gnd[i]['junk'])
        except:
            qgndj = np.empty(0)

        # sorted positions of positive and junk images (0 based)
        pos  = np.arange(ranks.shape[0])[np.in1d(ranks[:,i], qgnd)]
        junk = np.arange(ranks.shape[0])[np.in1d(ranks[:,i], qgndj)]

        k = 0
        ij = 0
        if len(junk):
            # decrease positions of positives based on the number of
            # junk images appearing before them
            ip = 0
            while (ip < len(pos)):
                while (ij < len(junk) and pos[ip] > junk[ij]):
                    k += 1
                    ij += 1
                pos[ip] = pos[ip] - k
                ip += 1

        # compute ap
        ap = compute_ap(pos, len(qgnd))
        map = map + ap
        aps[i] = ap

        # compute precision @ k
        pos += 1 # get it to 1-based
        for j in np.arange(len(kappas)):
            kq = min(max(pos), kappas[j]); 
            prs[i, j] = (pos <= kq).sum() / kq
        pr = pr + prs[i, :]

    map = map / (nq - nempty)
    pr = pr / (nq - nempty)

    return map, aps, pr, prs, nempty

In [55]:
def compute_metrics(dataset, ranks, gnd, kappas=[1, 5, 10]):
    print(ranks.shape)
    
    # old evaluation protocol
    if dataset.startswith('classic'):
        map, aps, _, _ = compute_map(ranks, gnd)
        out = {'map': np.around(map*100, decimals=3)}
        print('>> {}: mAP {:.2f}'.format(dataset, out['map']))

    # new evaluation protocol
    elif dataset.startswith('revisited'):
        
        #gnd_t = []
        #for i in range(len(gnd)):
        #    g = {}
        #    g['ok'] = np.concatenate([gnd[i]['easy']])
        #    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
        #    gnd_t.append(g)
        #mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t, kappas)

        #gnd_t = []
        #for i in range(len(gnd)):
        #    g = {}
        #    g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
        #    g['junk'] = np.concatenate([gnd[i]['junk']])
        #    gnd_t.append(g)
        #mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t, kappas)

        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH, nempty = compute_map(ranks, gnd_t, kappas)


        out = {
            #'E_map': np.around(mapE*100, decimals=2),
            #'M_map': np.around(mapM*100, decimals=2),
            'H_map': np.around(mapH*100, decimals=2),
            #'E_mp':  np.around(mprE*100, decimals=2),
            #'M_mp':  np.around(mprM*100, decimals=2),
            'H_mp':  np.around(mprH*100, decimals=2),
        }

        #print('>> {}: mAP E: {}, M: {}, H: {}'.format(dataset, out['E_map'], out['M_map'], out['H_map']))
        print('>> {}: mAP H: {}'.format(dataset, out['H_map']))
        #print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(dataset, kappas, out['E_mp'], out['M_mp'], out['H_mp']))
        print('>> {}: mP@k{} H: {}'.format(dataset, kappas, out['H_mp']))

    return out, mapH, apsH, mprH, prsH, nempty

In [56]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))
len(gnd_data['gnd'])

70

In [58]:
out, mapH, apsH, mprH, prsH, nempty  =compute_metrics('revisited', nn_inds.T, gnd_data['gnd'], kappas=[1,5,10])

(4993, 70)
>> revisited: mAP H: 45.11
>> revisited: mP@k[1, 5, 10] H: [85.71 72.29 60.14]


In [59]:
mapH, mprH, nempty

(0.45108796000449053, array([0.85714286, 0.72285714, 0.60142857]), 0)

In [88]:
nn_inds.T[:,10]

array([3829, 3152,  607, ..., 3800, 4026, 3246])

In [103]:
len(gnd_data['gnd'][10]['hard'])

10

In [105]:
qrels_dict = {}
run_dict = {}
for i in range(nn_inds.shape[0]):
    q_str = 'q_' + str(i)
    
    qrels_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in gnd_data['gnd'][i]['hard']])
    run_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in nn_inds.T[:,i]])


In [106]:
nn_inds.shape[0]

70

In [107]:
from ranx import Qrels, Run

qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [110]:
from ranx import evaluate

evaluate(qrels, run, ["map", "map@1", "map@5", "map@10", "mrr", "mrr@1", "mrr@5", "mrr@10", "f1@1"])
#>>> {"map@5": 0.6416, "mrr": 0.75}

{'map': 0.20006625392442232,
 'map@1': 0.030317794838342786,
 'map@5': 0.0949393884202708,
 'map@10': 0.10773165740047304,
 'mrr': 0.23435308009830536,
 'mrr@1': 0.07142857142857142,
 'mrr@5': 0.19309523809523807,
 'mrr@10': 0.21769274376417233,
 'f1@1': 0.03179706230553688}